In [1]:
spark

In [2]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

import time
# from utils import (
#     _to_dict,
#     _donut,
#     _pie,
#     _line_graph,
#     _other_fields,
#     _exitcode_info,
#     _better_label
# )
from datetime import datetime, date, timedelta
from pyspark.sql.functions import (
    col,
    lit,
    when,
    sum as _sum,
    count as _count,
    first,
    date_format,
    from_unixtime
)
import numpy as np
import pandas as pd
from pyspark.sql.types import (
    StructType,
    LongType,
    StringType,
    StructField,
    DoubleType,
    IntegerType,
)
# spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")


### Prepare condor file name/configuration

In [3]:
def _get_schema():
    return StructType(
        [
            StructField(
                "data",
                StructType(
                    [
                        StructField("RecordTime", LongType(), nullable=False),
                        StructField("CMSPrimaryDataTier", StringType(), nullable=True),
                        StructField("Status", StringType(), nullable=True),
                        StructField("WallClockHr", DoubleType(), nullable=True),
                        StructField("CoreHr", DoubleType(), nullable=True),
                        StructField("CpuTimeHr", DoubleType(), nullable=True),
                        StructField("Type", StringType(), nullable=True),
                        StructField("CRAB_DataBlock", StringType(), nullable=True),
                        StructField("GlobalJobId", StringType(), nullable=False),
                        StructField("ExitCode", LongType(), nullable=True),
                        StructField("CRAB_Workflow", StringType(), nullable=True),
                        StructField("CommittedCoreHr", StringType(), nullable=True),
                        StructField("CommittedWallClockHr", StringType(), nullable=True),
                    ]
                ),
            ),
        ]
    )

In [4]:
_DEFAULT_HDFS_FOLDER = "/project/monitoring/archive/condor/raw/metric"

In [5]:
# # Check available files 
# !hdfs dfs -ls /project/monitoring/archive/condor/raw/metric/2023/07/08

In [6]:
def get_candidate_files(start_date, end_date, spark, base=_DEFAULT_HDFS_FOLDER):
    st_date = start_date - timedelta(days=0)
    ed_date = end_date + timedelta(days=0)
    days = (ed_date - st_date).days
    pre_candidate_files = [
        "{base}/{day}{{,.tmp}}".format(
            base=base, day=(st_date + timedelta(days=i)).strftime("%Y/%m/%d")
        )
        for i in range(0, days)
    ]
    sc = spark.sparkContext
    
    candidate_files = [
        f"{base}/{(st_date + timedelta(days=i)).strftime('%Y/%m/%d')}"
        for i in range(0, days)
    ]
    FileSystem = sc._gateway.jvm.org.apache.hadoop.fs.FileSystem
    URI = sc._gateway.jvm.java.net.URI
    Path = sc._gateway.jvm.org.apache.hadoop.fs.Path
    fs = FileSystem.get(URI("hdfs:///"), sc._jsc.hadoopConfiguration())
    # FIXME
    candidate_files = [url for url in candidate_files if fs.globStatus(Path(url))]
    print("No. of Consisted files:", len(candidate_files))
    return candidate_files

#     all_candidate_files = []
#     candidate_files = [
#         f"{base}/{(st_date + timedelta(days=i)).strftime('%Y/%m/%d')}"
#         for i in range(0, days)
#     ]
    
#     URI           = sc._gateway.jvm.java.net.URI
#     Path          = sc._gateway.jvm.org.apache.hadoop.fs.Path
#     FileSystem    = sc._gateway.jvm.org.apache.hadoop.fs.FileSystem
#     Configuration = sc._gateway.jvm.org.apache.hadoop.conf.Configuration
#     fs = FileSystem.get(URI("hdfs:///"), Configuration())

#     for fileNames in candidate_files:
#         status = fs.listStatus(Path(fileNames))
#         candidate_files_day_i = [
#             str(fileStatus.getPath()).replace('hdfs://analytix', '')
#             for fileStatus in status
#         ]
#         all_candidate_files.extend(candidate_files_day_i)
#     print("Files Directory:", candidate_files, "\nNo. of Consisted files:", len(all_candidate_files))
#     return all_candidate_files

def group_files(files, n=16):
    # Yield successive n-sized
    # chunks from files
    all_group = []
    for i in range(0, len(files), n):
        all_group.append(files[i:i+n])
    print("There are", len(all_group), "chunks of files")
    return all_group

## load dataset

In [7]:
schema = _get_schema()
start_date = datetime(2023, 8, 10)
end_date = datetime(2023, 8, 11)

In [8]:
candidate_files = get_candidate_files(start_date, end_date, spark, base=_DEFAULT_HDFS_FOLDER)
candidate_files

No. of Consisted files: 1


['/project/monitoring/archive/condor/raw/metric/2023/08/10']

### Prepare CRAB data file name

In [9]:
TODAY = str(end_date)[:10]
wa_date = TODAY
HDFS_CRAB_part = f'/project/awg/cms/crab/tasks/{wa_date}/'

### Get raw data from condor raw

In [10]:
spark.conf.set("spark.sql.session.timeZone", "UTC")

crab_df = spark.read.format('avro').load(HDFS_CRAB_part)
crab_df = crab_df.select('TM_TASKNAME', 'TM_IGNORE_LOCALITY')

In [25]:
condor_df = (
        spark.read.option("basePath", _DEFAULT_HDFS_FOLDER)
        .json(
            candidate_files,
            schema=schema,
        ).select("data.*")
        .filter(
            f"""Status IN ('Completed')
            AND Type IN ('analysis')
            AND RecordTime >= {start_date.timestamp() * 1000}
            AND RecordTime < {end_date.timestamp() * 1000}
            """
        )
        .drop_duplicates(["GlobalJobId"]).cache()
    ) 
condor_df.write.mode('overwrite').parquet("/cms/users/eatthaph/condor_vir_data" ,compression='zstd')
condor_df = spark.read.format('parquet').load('/cms/users/eatthaph/condor_vir_data')
# condor_df.count()

23/08/16 13:48:02 WARN CacheManager: Asked to cache already cached data.


In [27]:
result_df = condor_df.join(crab_df, crab_df["TM_TASKNAME"] == condor_df["CRAB_Workflow"])\
    .select('RecordTime', 'CMSPrimaryDataTier', 'WallClockHr', 'CoreHr', 'CpuTimeHr', 'ExitCode'
            , "CRAB_DataBlock", "TM_IGNORE_LOCALITY", "GlobalJobId", "CommittedCoreHr", "CommittedWallClockHr")
docs = result_df.toPandas()

In [ ]:
len(docs)

In [19]:
# def spark_exec(candidate_files):
#     condor_df = (
#             spark.read.option("basePath", _DEFAULT_HDFS_FOLDER)
#             .json(
#                 candidate_files,
#                 schema=schema,
#             ).select("data.*")
#             .filter(
#                 f"""Status IN ('Completed')
#                 AND Type IN ('analysis')
#                 AND RecordTime >= {start_date.timestamp() * 1000}
#                 AND RecordTime < {end_date.timestamp() * 1000}
#                 """
#             )
#             .drop_duplicates(["GlobalJobId"]).cache()
#         ) 
#     condor_df.write.mode('overwrite').parquet("/cms/users/eatthaph/condor_vir_data" ,compression='zstd')
#     condor_df = spark.read.format('parquet').load('/cms/users/eatthaph/condor_vir_data')
#     result_df = condor_df.join(crab_df, crab_df["TM_TASKNAME"] == condor_df["CRAB_Workflow"])\
#         .select('RecordTime', 'CMSPrimaryDataTier', 'WallClockHr', 'CoreHr', 'CpuTimeHr', 'ExitCode'
#                 , "CRAB_DataBlock", "TM_IGNORE_LOCALITY", "GlobalJobId", "CommittedCoreHr", "CommittedWallClockHr")
#     sub_docs = result_df.toPandas()
#     return sub_docs

# def loop_excute(candidate_files, initial_n=len(candidate_files)):
#     r = 0
#     n = initial_n
#     df_list = []
#     file_chunk = group_files(candidate_files, n)
#     while len(file_chunk)!=0 and r<10:
#         print("=================================\n round :", r+1, "\n=================================")
#         df_err_list = []
#         for i, chunk in enumerate(file_chunk):
#             print("=================================\n", i+1, "out of", len(file_chunk), "\n=================================")
#             try:
#                 df_list.append(spark_exec(chunk))
#             except Exception as ex:
#                 print("=====", ex)
#                 df_err_list.extend(chunk)
# #         if n != 1:
# #             n = n//2
#         file_chunk = group_files(df_err_list, n)
#         r += 1
#         print("")
#     print("Fail excuted files :", df_err_list)
#     return df_list

In [1]:
# useful_df = loop_excute(candidate_files)
# df_list = spark_exec(candidate_files)

In [15]:
docs = docs.to_dict('records')

In [16]:
for i in range(len(docs)):
    if docs[i]['CRAB_DataBlock'] == 'MCFakeBlock':
        docs[i]['CRAB_Type'] = 'PrivateMC'
    else:
        docs[i]['CRAB_Type'] = 'Analysis'

In [17]:
docs[:5]

[{'RecordTime': 1692101192000,
  'CMSPrimaryDataTier': 'MINIAODSIM',
  'WallClockHr': 0.12361111111111112,
  'CoreHr': 0.12361111111111112,
  'CpuTimeHr': 0.0022222222222222222,
  'ExitCode': 8020,
  'CRAB_DataBlock': '/WWTo4Q_4f_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIISummer20UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v3/MINIAODSIM#eb5a0cbd-6c43-492c-ac21-4318775aee3b',
  'TM_IGNORE_LOCALITY': 'F',
  'GlobalJobId': 'crab3@vocms0155.cern.ch#98631116.0#1692100543',
  'CommittedCoreHr': '0.12361111111111112',
  'CommittedWallClockHr': '0.12361111111111112',
  'CRAB_Type': 'Analysis'},
 {'RecordTime': 1692099933000,
  'CMSPrimaryDataTier': 'MINIAODSIM',
  'WallClockHr': 0.12166666666666667,
  'CoreHr': 0.12166666666666667,
  'CpuTimeHr': 0.004722222222222222,
  'ExitCode': 8020,
  'CRAB_DataBlock': '/WWTo4Q_4f_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIISummer20UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v3/MINIAODSIM#eb5a0cbd-6c43-492c-ac21-4318775aee3b',
  'TM_IGNORE_LOCALITY':

In [18]:
import osearch

In [19]:
def get_index_schema():
    return {
        "settings": {"index": {"number_of_shards": "1", "number_of_replicas": "1"}},
        "mappings": {
            "properties": {
                "RecordTime": {"format": "epoch_millis", "type": "date"},
                "CMSPrimaryDataTier": {"ignore_above": 2048, "type": "keyword"},
                "GlobalJobId": {"ignore_above": 2048, "type": "keyword"},
                "WallClockHr": {"type": "long"},
                "CoreHr": {"type": "long"},
                "CpuTimeHr": {"type": "long"},
                "ExitCode": {"ignore_above": 2048, "type": "keyword"},
                "TM_IGNORE_LOCALITY": {"ignore_above": 2048, "type": "keyword"},
                "CRAB_Type": {"ignore_above": 2048, "type": "keyword"},
                "CRAB_DataBlock": {"ignore_above": 2048, "type": "keyword"},
                "CommittedCoreHr": {"type": "long"}, 
                "CommittedWallClockHr": {"type": "long"},
            }
        }
    }

In [20]:
_index_template = 'crab-condor-ekong'
client = osearch.get_es_client("es-cms1.cern.ch/es", 'secret_opensearch.txt', get_index_schema())
# index_mod="": 'test-foo', index_mod="Y": 'test-foo-YYYY', index_mod="M": 'test-foo-YYYY-MM', index_mod="D": 'test-foo-YYYY-MM-DD',
idx = client.get_or_create_index(timestamp=time.time(), index_template=_index_template, index_mod="M")
client.send(idx, docs, metadata=None, batch_size=10000, drop_nulls=False)

/eos/user/e/eatthaph/.local/lib/python3.9/site-packages/opensearchpy/connection/http_urllib3.py:199: UserWarning: Connecting to https://es-cms1.cern.ch:443 using SSL with verify_certs=False is insecure.
  warnings.warn(
/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es-cms1.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es-cms1.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-ce

/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es-cms1.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es-cms1.cern.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es-cms1.cern.ch'. Adding certificate verification is strongly advised. S